<a href="https://colab.research.google.com/github/mamud1977/bootup/blob/main/NN_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install tensorflow-gpu==2.8.4


In [2]:
# ! pip install tqdm

In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2024-09-14 19:16:13--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.121.207, 209.85.145.207, 142.250.125.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.121.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   192MB/s    in 0.3s    

2024-09-14 19:16:13 (192 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [11]:
import os
import zipfile as zf
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data preprocessing


In [36]:
files = zf.ZipFile('./cats_and_dogs_filtered.zip', "r").extractall(  )
zf.ZipFile('./cats_and_dogs_filtered.zip', "r").close()

In [21]:
dataset_path = "./cats_and_dogs_filtered/"
train_dir = os.path.join(dataset_path, "train")
validation_dir = os.path.join(dataset_path, "validation")

In [33]:
data_gen_train = ImageDataGenerator(rescale=1/255)
data_gen_valid = ImageDataGenerator(rescale=1/255)

In [34]:
train_generator = data_gen_train.flow_from_directory(train_dir,
                                                     batch_size=20,
                                                     class_mode='binary',
                                                     target_size=(128,128))

Found 2000 images belonging to 2 classes.


In [35]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir,
                                                     batch_size=20,
                                                     class_mode='binary',
                                                     target_size=(128,128))

Found 1000 images belonging to 2 classes.


## Building the model

In [23]:
img_shape = (128,128,3)

base_model = tf.keras.applications.MobileNetV2(input_shape=img_shape,
                                               include_top=False,
                                               weights='imagenet')


9420800/9406464 [==============================] - 0s 0us/step


In [24]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][

In [25]:
base_model.trainable = False

In [26]:
base_model.output

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

In [27]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [28]:
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [29]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

In [30]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [31]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][0]']           

In [32]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


## Training the model

In [37]:
model.fit_generator(train_generator,
                    epochs=5,
                    validation_data=valid_generator)

<ipython-input-37-d5b91343d936>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


Epoch 1/5
100/100 [==============================] - 43s 384ms/step - loss: 0.6420 - accuracy: 0.6450 - val_loss: 0.4516 - val_accuracy: 0.8110
Epoch 2/5
100/100 [==============================] - 38s 385ms/step - loss: 0.3635 - accuracy: 0.8570 - val_loss: 0.2820 - val_accuracy: 0.8970
Epoch 3/5
100/100 [==============================] - 38s 384ms/step - loss: 0.2460 - accuracy: 0.9195 - val_loss: 0.2106 - val_accuracy: 0.9200
Epoch 4/5
100/100 [==============================] - 38s 383ms/step - loss: 0.1934 - accuracy: 0.9365 - val_loss: 0.1737 - val_accuracy: 0.9360
Epoch 5/5
100/100 [==============================] - 37s 375ms/step - loss: 0.1644 - accuracy: 0.9450 - val_loss: 0.1535 - val_accuracy: 0.9420


In [38]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

<ipython-input-38-a86cc1f7f1e2>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)


In [41]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))
print("Loss after transfer learning: {}".format(valid_loss))

Accuracy after transfer learning: 0.9419999718666077
Loss after transfer learning: 0.15354618430137634


##Fine tuning

In [42]:
base_model.trainable = True

In [43]:
print("Number of layers in the base model: {}".format(len(base_model.layers)))

Number of layers in the base model: 154


In [45]:
fine_tune_at = 120

for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

In [46]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [47]:
model.fit_generator(train_generator,
                    epochs=5,
                    validation_data=valid_generator)

Epoch 1/5


<ipython-input-47-d5b91343d936>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


100/100 [==============================] - 62s 582ms/step - loss: 0.1620 - accuracy: 0.9385 - val_loss: 0.0651 - val_accuracy: 0.9790
Epoch 2/5
100/100 [==============================] - 59s 592ms/step - loss: 0.0540 - accuracy: 0.9795 - val_loss: 0.0868 - val_accuracy: 0.9760
Epoch 3/5
100/100 [==============================] - 59s 588ms/step - loss: 0.0331 - accuracy: 0.9865 - val_loss: 0.2600 - val_accuracy: 0.9360
Epoch 4/5
100/100 [==============================] - 58s 581ms/step - loss: 0.0214 - accuracy: 0.9955 - val_loss: 0.3801 - val_accuracy: 0.9320
Epoch 5/5
100/100 [==============================] - 57s 574ms/step - loss: 0.0247 - accuracy: 0.9930 - val_loss: 0.3827 - val_accuracy: 0.9280


In [48]:
alid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

<ipython-input-48-2d2bca2c56e8>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  alid_loss, valid_accuracy = model.evaluate_generator(valid_generator)


In [49]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))
print("Loss after transfer learning: {}".format(valid_loss))

Accuracy after transfer learning: 0.9279999732971191
Loss after transfer learning: 0.15354618430137634
